In [ ]:
!pip install feedparser 
!pip install newspaper3k 
!pip install konlpy 
!pip install bs4 
!sudo apt-get install -y fonts-nanum 
!sudo fc-cache -fv 
!rm ~/.cache/matplotlib -rf

In [5]:
import feedparser
from newspaper import Article 
from konlpy.tag import Okt 
from collections import Counter 
from operator import eq 
import numpy as np 
from bs4 import BeautifulSoup

In [6]:
urls = ['https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=01&plink=RSSREADER']

In [20]:
def crawl_rss(urls):
  arr_rss= []
  for url in urls:
    print("[Crawl RSS] ", url)
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries:
      arr_rss.append({'title': p.title, 'link':p.link})
  return arr_rss

def crawl_article(url, language='ko'):
  print("Crawl Article] ", url)
  val_article = Article(url, language=language)
  val_article.download()
  val_article.parse()
  return val_article.title, val_article.text

def get_keywords(text, nKeywords=50):
  num_unique_words = 0
  num_most_freq = 0
  ranking = 0
  spliter = Okt()
  nouns = spliter.nouns(text)
  count = Counter(nouns)
  return_list = []
  for n, c in count.most_common(nKeywords):
    ranking = ranking + 1
    temp = {'keyword' : n, 'count':c, 'ranking' : ranking}
    return_list.append(temp)
    num_unique_words = num_unique_words + 1
    if num_unique_words == 1:
      num_most_freq = c
  return num_unique_words, num_most_freq, return_list

def Howmanywords(request, keyword):
  nWords = 0
  nRanking = 0
  for n in keyword:
    noun = n['keyword']
    count = n['count']
    rank = n['ranking']
    if eq(noun, request):
      nWords = count
      nRanking = rank
  return nWords, nRanking

In [21]:
# STARTING POINT 
article_list = crawl_rss(urls)
print(article_list)

for article in article_list:
  _, text = crawl_article(article['link'])
  article['text'] = text

print('[Parsing Text]')
for article in article_list:
  num_unique_words, num_most_freq, keywords = get_keywords(article['text'], 1000)
  article['keywords'] = keywords
  article['num_unique_words'] = num_unique_words
  article['num_most_freq'] = num_most_freq

print('Enter your query :')
query = input()
for article in article_list:
  n, _ = Howmanywords(query, article['keywords'])
  if n!= 0:
    print('TF : ', n, article['title'], article['link'], '\n')

[Crawl RSS]  https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=01&plink=RSSREADER
[{'title': '문 대통령, NSC 긴급회의 주재…북 미사일 발사 관련', 'link': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1006688163&plink=RSSLINK&cooper=RSSREADER'}, {'title': '민주당 원내대표 후보, 박광온 박홍근 이원욱 최강욱 4명 압축', 'link': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1006688107&plink=RSSLINK&cooper=RSSREADER'}, {'title': "'수사지휘권 폐지' 놓고 법무부-윤 당선인 정면충돌", 'link': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1006687770&plink=RSSLINK&cooper=RSSREADER'}, {'title': '합참 "북, 장거리탄도미사일 1발 발사 추정"', 'link': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1006687996&plink=RSSLINK&cooper=RSSREADER'}, {'title': '청와대 회동 \'공전\'에…문 대통령 "윤 당선인 직접 판단하길"', 'link': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1006687769&plink=RSSLINK&cooper=RSSREADER'}, {'title': '5년 만에 카메라 앞에 선 박근혜…웃으며 한 첫 마디', 'link': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1006687765&plink=RSSLINK&cooper=RSSREADER'}, {'title': '"북한, 장거리 탄도미사일 1발 고각 발